In [ ]:
import pandas as pd

## Load

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
matrix_factorization = pd.read_csv('matrix_factorization.csv')
matrix_factorization.head()

## Benchmark model evaluation